In [13]:
import json
from pprint import pprint

In [22]:
my_json = \
"""
{
  
  "conv2d": {
     "filters": 32, 
     "kernel_size": [3, 3], 
     "activation": "relu", 
     "padding": "same", 
     "input_shape": [200, 200, 3]
     },
  
  "mp2": {
     "pool_size": [2, 2]
     }

}
"""

In [23]:
my_json

'\n{\n "conv2d": {\n     "filters": 32, \n     "kernel_size": [3, 3], \n     "activation": "relu", \n     "padding": "same", \n     "input_shape": [200, 200, 3]\n     }, \n "mp2": {\n     "pool_size": [2, 2]\n     }\n}\n'

In [24]:
inputs = json.loads(my_json)

In [25]:
inputs

{'conv2d': {'filters': 32,
  'kernel_size': [3, 3],
  'activation': 'relu',
  'padding': 'same',
  'input_shape': [200, 200, 3]},
 'mp2': {'pool_size': [2, 2]}}

In [31]:
symbol_table = {
    'conv2d' : 'model.add(Conv2D())',
    'mp2' : 'model.add(MaxPooling2D())',
    'dense': 'model.add(Dense())'
}

In [28]:
def get_imports():
    """
    imports the needed modules 
    """
    try:
        return \
        """
        import sys
        from matplotlib import pyplot
        from keras.utils import to_categorical
        from keras.models import Sequential
        from keras.layers import Conv2D
        from keras.layers import MaxPooling2D
        from keras.layers import Dense
        from keras.layers import Flatten
        from keras.optimizers import SGD
        from keras.preprocessing.image import ImageDataGenerator
        """
    except Exception as e:
        print(e)

In [30]:
def init_sequential():
    """
    initializes the model
    """
    try:
        get_imports()
        
        return 'model = Sequential()'
    except Exception as e:
        print(e)

In [49]:
def parse(inputs):
    """
    Parser which adds layers (mostly)
    """
    try:
        # init_sequential()
        input_dict = json.loads(inputs)
        print("\nGot these layers")
        generated_code = ''

        #TODO: Take care of indentation and stuff if making functions
        #TODO: Ordering in dict (work around -> ordered_dict)

        for layer in input_dict.keys():
            curr_layer = symbol_table[layer]
            print(curr_layer,'99999999999999999999')
            args = str(input_dict[layer])
            curr_layer = curr_layer[:-2] + args[1:-1] + curr_layer[-2:]
            generated_code += curr_layer + '\n'
        print()
        return generated_code
    except Exception as e:
        print(e)


In [50]:
print(parse(my_json))


Got these layers
model.add(Conv2D()) 99999999999999999999
model.add(MaxPooling2D()) 99999999999999999999

model.add(Conv2D('filters': 32, 'kernel_size': [3, 3], 'activation': 'relu', 'padding': 'same', 'input_shape': [200, 200, 3]))
model.add(MaxPooling2D('pool_size': [2, 2]))



## Work Pad ahead (no code below this)


_tip : take a "md" crash course to write good markdown code xD_   

## Json Schema

### Model Structure
- Use model.add() method
- Front end drop down for each layer

<!-- 
====================
padding [v]
        - same
        - opt2
        - opt3
====================
 -->


## Work Distribution


_Do the following steps for both torch and keras_


#### Preprocessing + Data input (Tanvi)
- os module
- raw images :D
- keras...
- data/
    /train
        /category1
        /category2
    /test
        /category1
        /category2

#### Preparation of Symbol Table and Schema

 

#### Adding layers

- _/

#### Compile and execute (Raj)

- 

